<a href="https://colab.research.google.com/github/matteraggi/FineTuningAI/blob/main/ProgettoTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup iniziale

**INSTALLS**

In [1]:
!pip install --upgrade transformers torch datasets peft tf-keras accelerate bitsandbytes trl evaluate radon zss torchvision
# IMPORTANTE! RIAVVIARE IL RUNTIME DOPO L'ESECUZIONE

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.6 MB/s eta 0:00:00
   ━━━

**IMPORTS**

In [ ]:
# Importing stock ml libraries
import pandas as pd
import torch
import gc
from tabulate import tabulate
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
import json
import random
from google.colab import files

**Svuota la cache**

In [2]:
# Garbace collect
gc.collect()
# Svuota la cache
torch.cuda.empty_cache()

**VARIABLES**

In [ ]:
# The model that you want to train from the Hugging Face hub
#model_name = "bigcode/starcoder2-3b"
#model_name = "openai-community/gpt2"
#model_name = "NousResearch/Llama-2-7b-hf"
#model_name = "deepcogito/cogito-v1-preview-llama-3B"
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# The instruction dataset to use
dataset_name = "bigcode/self-oss-instruct-sc2-exec-filter-50k"
# Fine-tuned model name
new_model = "Our_Finetuned_Model"

# Dimension of dataset subset used
dataset_range = 4000

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension. It determines the size and parameter count of the low-rank adaptation
lora_r = 64
# Alpha parameter for LoRA scaling factor that determines the impact of the low-rank matrices on the original model's output.
# Controls the overall strength of the low-rank adaptation.
lora_alpha = 2*lora_r  # "Often set to 2-4 times lora_r"
# Dropout probability for LoRA layers (considera aumento a 0.1-0.2)
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False  # "Double quantization can sometimes improve performance but increases complexity.  It's often left disabled initially."

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False  # CONFERMO CHE NON E' SUPPORTATO SU COLAB

# Number of training epochs
num_train_epochs = 3  # Sembra arrivare a un plateau già alla terza
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4  # Abbassare in caso di training più lunghi

# Batch size per training e per evaluation
per_device_train_batch_size = per_device_eval_batch_size = 2
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4 # Moltiplica la batch_size vera per ottenere quella simulata. Dinimuire per velocità, alzare per stabilità.

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3  # Stabilizza il training, ma rallentandolo. Range ottimale 0.1-0.5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.1  # Aumentare in caso di overfitting a 0.1-0.2

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1  # '-1' mantains num_train_epochs

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.05 # Aumentare a 0.1 in caso di instabilità eccessiva iniziale

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every tot updates steps
save_steps = 100
# Log every tot updates steps
logging_steps = 100
# Evaluation strategy ("no", "epochs", "steps")
eval_strategy = "steps" # unused
# Evaluate the model every tot steps (if the strategy is "steps")
eval_steps = 100        # unused

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use; tronca l'input a tot tokens
max_seq_length = 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False  # "Can improve efficiency if your dataset has many short sequences."

# Load the entire model on the GPU 0
device_map = {"": 0}

### Caricamento del modello di base (per il fine-tuning)

In [4]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)
# Il warning di sliding window è un false warning: https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/discussions/27

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  device_map=device_map
)

# Load base tokenizer
tokenizer_base = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
print(tokenizer_base.pad_token, tokenizer_base.eos_token)
tokenizer_base.pad_token = tokenizer_base.eos_token
print(tokenizer_base.pad_token, tokenizer_base.eos_token)
tokenizer_base.padding_side = 'right'

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

<｜end▁of▁sentence｜> <｜end▁of▁sentence｜>
<｜end▁of▁sentence｜> <｜end▁of▁sentence｜>


In [ ]:
# Ultime configurazioni per il modello
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1
if gradient_checkpointing:
  base_model.gradient_checkpointing_enable()  # Attiva gradient checkpointing per ridurre l'uso di memoria

# for name, module in base_model.named_modules():
#    print(name)

# Load LoRA configuration
peft_config = LoraConfig(
  lora_alpha=lora_alpha,
  lora_dropout=lora_dropout,
  r=lora_r,
  bias="none",
  task_type="CAUSAL_LM",
  #target_modules=["q_proj", "v_proj"] # also ["o_proj", "k_proj"] ?
)

print(base_model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps

### Caricamento del modello fine-tuned da GitHub (per la valutazione)

In [ ]:
#Da Git:
!apt-get install git
!rm -rf FineTuningAI
!git clone https://github.com/matteraggi/FineTuningAI.git

#Dal Drive (cambiare path manualmente):
#from google.colab import drive
#drive.mount('/content/drive')
#!cp -r /content/drive/MyDrive/checkpoint_800 /content/

# Percorso alla directory del modello
checkpoint_path = "/content/FineTuningAI/models/DS_Finetuned_6"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)
# Il warning di sliding window è un false warning: https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/discussions/27

# Load (partially) finetuned model
finetuned_model = AutoModelForCausalLM.from_pretrained(
  checkpoint_path,
  device_map=device_map,
  quantization_config=bnb_config,
)

# Load tokenizer from checkpoint
tokenizer_finetuned = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer_finetuned.pad_token = tokenizer_finetuned.eos_token
tokenizer_finetuned.padding_side = 'right'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Cloning into 'FineTuningAI'...
remote: Enumerating objects: 350, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 350 (delta 5), reused 5 (delta 1), pack-reused 334 (from 1)
Receiving objects: 100% (350/350), 587.01 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (193/193), done.
Updating files: 100% (59/59), done.


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# Ultime configurazioni per il modello
finetuned_model.config.use_cache = False
finetuned_model.config.pretraining_tp = 1
if gradient_checkpointing:
  finetuned_model.gradient_checkpointing_enable()  # Attiva gradient checkpointing per ridurre l'uso di memoria

# for name, module in finetuned_model.named_modules():
#    print(name)

# Load LoRA configuration
peft_config = LoraConfig(
  lora_alpha=lora_alpha,
  lora_dropout=lora_dropout,
  r=lora_r,
  bias="none",
  task_type="CAUSAL_LM",
  #target_modules=["q_proj", "v_proj"] # o ["o_proj", "k_proj"] ?
)

print(finetuned_model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1536, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=1536, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=1536, o

### Caricamento e tokenizzazione dataset

**DATASET LOAD AND PREPROCESSING**

In [15]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
# Use only a random subset of the dataset 'dataset_range'-wide
dataset = dataset.shuffle(seed=42).select(range(dataset_range))

# # Split the subset in training and validation
# split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = split_dataset["train"]
# val_dataset = split_dataset["test"]

# Use the full dataset for training only
train_dataset = dataset
# Per evitare OOM, è stata rimossa la validazione. Alla fine il codice verrà sempre valutato con humaneval.

In [17]:
# Funzione che ritaglia solo la funzione dal testo di response

import re

def extract_code_from_response(text):
    # Cerca il primo blocco ```python ... ```
    match = re.search(r"```(?:python)?\n(.*?)```", text, re.DOTALL) # Tre backtick, 'python' opzionale, '\n', prendi il blocco, tre backtick. re.DOTALL per proseguire alle nuove linee
    if match:
        return match.group(1).strip() # Il contenuto catturato, stripped
    else:
        print('fallback!')  # DEBUGGING. Non dovrebbe printare nulla
        return text.strip() # Fallback

def clean_response(example):
    example["response"] = extract_code_from_response(example["response"])
    return example

dataset = dataset.map(clean_response)

In [19]:
# Check dataset after cutting functions from responses

#print dataset columns
print(dataset.column_names)
#print dataset example
print(dataset[0])
#print dataset example with instruction and seed
print(f"Instruction: {dataset[1]['instruction']}")
print(f"Response: {dataset[1]['response']}")

['fingerprint', 'sha1', 'seed', 'response', 'concepts', 'prompt', 'instruction', 'id']
{'fingerprint': None, 'sha1': 'a1ea8d1a52a57874b576afc6a4c45e7d624d409e', 'seed': 'def box_enum(typ, val, c):\n    """\n    Fetch an enum member given its native value.\n    """\n    valobj = c.box(typ.dtype, val)\n    # Call the enum class with the value object\n    cls_obj = c.pyapi.unserialize(c.pyapi.serialize_object(typ.instance_class))\n    return c.pyapi.call_function_objargs(cls_obj, (valobj,))', 'response': 'def serialize_objects(objects):\n    """\n    Serialize a list of objects into a list of dictionaries.\n    Each dictionary represents the object\'s class name and attributes.\n    """\n    serialized_objects = []\n    for obj in objects:\n        serialized_object = {\n            "class_name": obj.__class__.__name__,\n            "attributes": {}\n        }\n        for attr_name, attr_value in obj.__dict__.items():\n            attr_dict = serialize_value(attr_value)\n            seri

In [20]:
from statistics import mean, median

lengths = []

for ex in dataset:
    # Ricostruisci l'input esattamente come nel preprocessing
    full_text = f"{ex['instruction'].strip()}\n{ex['response'].strip()}"  # strip è opzionale
    # Tokenizzazione completa con special tokens (esattamente come nel preprocessing)
    input_ids = tokenizer_base(full_text, truncation=False)["input_ids"]
    #input_ids = tokenizer_finetuned(full_text, truncation=False)["input_ids"]
    lengths.append(len(input_ids))

# Analisi
max_len = max(lengths)
avg_len = mean(lengths)
med_len = median(lengths)
over_512 = sum(l > 512 for l in lengths) / len(lengths) * 100
over_1024 = sum(l > 1024 for l in lengths) / len(lengths) * 100
over_2048 = sum(l > 2048 for l in lengths) / len(lengths) * 100

print(f"Lunghezza massima: {max_len} token")
print(f"Lunghezza media: {avg_len:.2f} token")
print(f"Lunghezza mediana: {med_len} token")
print(f"Percentuale > 512 token: {over_512:.2f}%")
print(f"Percentuale > 1024 token: {over_1024:.2f}%")
print(f"Percentuale > 2048 token: {over_2048:.2f}%")

Lunghezza massima: 1340 token
Lunghezza media: 192.82 token
Lunghezza mediana: 174.0 token
Percentuale > 512 token: 1.25%
Percentuale > 1024 token: 0.10%
Percentuale > 2048 token: 0.00%


In [21]:
# Funzione di preprocessing degli input
def preprocess_function(examples):
    input_ids_list = []
    label_ids_list = []
    attention_mask_list = []

    for instr, resp in zip(examples["instruction"], examples["response"]):
        prompt = instr.strip() + "\n" # Aggiungi uno /n alla fine di ogni insturction
        completion = resp.strip()     # In questo dataset lo strip è ridondante, ma non fa male

        # tokenizzazione separata di prompt e codice, senza token speciali
        prompt_ids = tokenizer_base(prompt, add_special_tokens=False)["input_ids"]
        completion_ids = tokenizer_base(completion, add_special_tokens=False)["input_ids"]

        # Li uniamo e 'svalutiamo' l'istruzione stessa per fare in modo che il modello impari solo a generare codice
        input_ids = prompt_ids + completion_ids
        labels = [-100] * len(prompt_ids) + completion_ids  # -100 label mask standard di huggingface per escludere token

        # Padding/truncation
        if len(input_ids) > max_seq_length:     # Tronca eventuali input troppo lunghi
            input_ids = input_ids[:max_seq_length]
            labels = labels[:max_seq_length]
        else:                                   # Aggiungi padding per arrivare a maxseq (e svaluta i token di padding)
            pad_len = max_seq_length - len(input_ids)
            input_ids += [tokenizer_base.pad_token_id] * pad_len
            labels += [-100] * pad_len

        attention_mask = [1 if token != tokenizer_base.pad_token_id else 0 for token in input_ids] # 1 per token validi, 0 per padding (in DS, padding=eos)

        input_ids_list.append(input_ids)
        label_ids_list.append(labels)
        attention_mask_list.append(attention_mask)

    return {
        "input_ids": input_ids_list,  # Frasi tokenizzate, input del modello
        "labels": label_ids_list, # Maschera corretta per supervisionare solo la parte target
        "attention_mask": attention_mask_list # Maschera per nascondere i padding
    }

# Applica il preprocessing
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
#tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

### Training (Fine-tuning)

In [22]:
# Set training parameters
training_arguments = TrainingArguments(
  output_dir=output_dir,
  num_train_epochs=num_train_epochs,
  per_device_train_batch_size=per_device_train_batch_size,
  gradient_accumulation_steps=gradient_accumulation_steps,
  optim=optim,
  # eval_strategy=eval_strategy,  #eval
  # eval_steps=eval_steps,        #eval
  save_steps=save_steps,
  logging_steps=logging_steps,
  learning_rate=learning_rate,
  weight_decay=weight_decay,
  fp16=fp16,
  bf16=bf16,
  max_grad_norm=max_grad_norm,
  max_steps=max_steps,
  warmup_ratio=warmup_ratio,
  group_by_length=group_by_length,
  lr_scheduler_type=lr_scheduler_type,
  report_to="tensorboard",
  gradient_checkpointing=gradient_checkpointing,
  label_names=["labels"], # Risolve: No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
)

# Train the model with the modified configuration
trainer = SFTTrainer(
  model=base_model,
  train_dataset=tokenized_train_dataset,
  # eval_dataset=tokenized_val_dataset,  #eval
  peft_config=peft_config,
  args=training_arguments,
)

Truncating train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [23]:
# Start training

# Training da zero
trainer.train()

# Training se già parzialmente allenato
#trainer.train(resume_from_checkpoint=checkpoint_path)

Step,Training Loss
100,1.665300
200,0.229600
300,0.224300
400,0.216100
500,0.217400
600,0.208700
700,0.206100
800,0.209900
900,0.201500
1000,0.209700


TrainOutput(global_step=1500, training_loss=0.3051934852600098, metrics={'train_runtime': 10163.933, 'train_samples_per_second': 1.181, 'train_steps_per_second': 0.148, 'total_flos': 1.14457602686976e+17, 'train_loss': 0.3051934852600098})

### Salvataggio del modello / checkpoint

**Salva il modello**

In [24]:
# Save the fine-tuned model
#trainer.model.save_pretrained(new_model)
trainer.save_model("./results/DS_Finetuned_6")

**Salva il checkpoint/modello pushandolo su GitHub**

In [ ]:
from google.colab import userdata
!apt-get install git
!git config --global user.email {userdata.get('GitEmail')}
!git config --global user.name {userdata.get('GitUsername')}
!git clone https://github.com/matteraggi/FineTuningAI.git
!cd FineTuningAI  # Go to the *existing* FineTuningAI directory
!mv results/DS_Finetuned_6 FineTuningAI/models/DS_Finetuned_6/
!cd FineTuningAI && git add models/DS_Finetuned_6
!cd FineTuningAI && git commit -m "Saved 'DS_Finetuned_6', uses cut responses."
!git config --global credential.helper store
!cd FineTuningAI && git push https://{userdata.get('PAT')}@github.com/matteraggi/FineTuningAI.git main

**Salva il checkpoint/modello su Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/results/DS_Finetuned_6 /content/drive/MyDrive/

### Valutazione HumanEval

**Caricare il benchmark HumanEval**
Dopo il fine-tuning, devi confrontare il tuo modello con uno pre-addestrato (StarCoder2 senza fine-tuning) e con il modello fine-tunato.

In [ ]:
gc.collect()
torch.cuda.empty_cache()
#del non_finetuned_model

In [ ]:
SYSTEM_PROMPT = "Generate Python code for the following task:\n"
#SYSTEM_PROMPT = """Complete the following python function as described in its description.
#                    In said description, there are also lines beginning with >>> that symbolize execution examples, useful for understanding how the function should work.\n"""

In [ ]:
import torch
import json
import gc
import random
import pandas as pd
from tabulate import tabulate
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Configurazione
base_model_name = "bigcode/starcoder2-3b"   # Usiamo starcoder per il confronto
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device in use:", device) #DEBUG

# Caricamento del dataset HumanEval e selezione casuale di 10 esempi
humaneval = load_dataset("openai/openai_humaneval", split="test")
sampled_humaneval = random.sample(list(humaneval), 10)

# Configurazione quantizzazione     # Rimosso 04/05/2025
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)  # Aggiunto 04/05/2025
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)

# #DEBUGGING
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Caricamento del modello base
non_finetuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, device_map=device_map, quantization_config=bnb_config  #cambiata device_map da '"auto"' a 'device_map'. Cambiata quantizzazione come visto sopra.
)
non_finetuned_model.eval()

# Load starcoder (base model) tokenizer
tokenizer_base = AutoTokenizer.from_pretrained(base_model_name)
tokenizer_base.pad_token = tokenizer_base.eos_token
tokenizer_base.padding_side = 'right'

# Funzione per generare codice con il prompt modificato
def generate_code(model, prompt, tokenizer):
    # Aggiungiamo un contesto iniziale per istruire il modello correttamente
    full_prompt = SYSTEM_PROMPT + prompt
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200, # era 200
            do_sample=True,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Rimuovi il prompt iniziale dall'output generato (pezzo di codice nuovo! 04/05/2025)
    generated_tokens = outputs[0][input_length:]
    return tokenizer.decode(generated_tokens, skip_special_tokens=True)


# Valutazione dei modelli
base_results = []
fine_tuned_results = []

for i, example in enumerate(sampled_humaneval):
    prompt = example['prompt']

    base_code = generate_code(non_finetuned_model, prompt, tokenizer_base)
    base_results.append({"prompt": prompt, "code": base_code})

    fine_tuned_code = generate_code(finetuned_model, prompt, tokenizer_finetuned)
    fine_tuned_results.append({"prompt": prompt, "code": fine_tuned_code})

    print(f"Processed example {i+1}/10")
    gc.collect()
    torch.cuda.empty_cache()

# Salvataggio dei risultati in JSON
results = {"base_results": base_results, "fine_tuned_results": fine_tuned_results}

with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)

# Creazione e stampa tabella
df = pd.DataFrame([
    {"Prompt": b['prompt'], "Non-Finetuned Code": b['code'], "Finetuned Code": f['code']}
    for b, f in zip(base_results, fine_tuned_results)
])

files.download("model_results.json")

print(tabulate(df, headers="keys", tablefmt="grid", numalign="left", stralign="left"))

device in use: cuda


README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

Processed example 1/10
Processed example 2/10
Processed example 3/10
Processed example 4/10
Processed example 5/10
Processed example 6/10
Processed example 7/10
Processed example 8/10
Processed example 9/10
Processed example 10/10
+----+-------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | Prompt                                                                                                | Non-Finetuned Code                                                                                                                                               

In [ ]:
# Test con prompt semplice
import torch
import json
import gc
import random
import pandas as pd
from tabulate import tabulate
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Configurazione
base_model_name = "bigcode/starcoder2-3b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Caricamento del dataset HumanEval e selezione casuale di 10 esempi
humaneval = load_dataset("openai/openai_humaneval", split="test")
sampled_humaneval = random.sample(list(humaneval), 10)

# Configurazione quantizzazione
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Caricamento del modello base
non_finetuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, device_map="auto", quantization_config=quantization_config
).eval()


def generate_code(model, prompt):
    # Istruzione chiara: solo codice, nessuna spiegazione o testo extra
    instruction = "Complete the Python function.\n\n"
    full_prompt = prompt

    inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,  # Disabilita il campionamento
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,  # Usa eos_token come pad_token
        )

    # Decodifica l'output
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Estrai solo la parte che ci interessa, ossia il codice
    # Eliminare tutto prima del codice effettivo (per esempio, "Solution" e numeri casuali)
    start_index = full_output.find("def ")  # Trova la definizione della funzione
    cleaned_output = full_output[start_index:].strip()

    # Restituisci il codice "pulito"
    return cleaned_output

test_prompt = "def add(a, b):\n    \"\"\"Add two numbers.\"\"\"\n"
print("Base model:", generate_code(non_finetuned_model, test_prompt))
print("Fine-tuned:", generate_code(finetuned_model, test_prompt))

In [ ]:
import json

# Assuming base_results and fine_tuned_results are your arrays
results = {
    "base_results": base_results,
    "fine_tuned_results": fine_tuned_results
}

# Save as JSON
with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)



In [ ]:
import pandas as pd
from tabulate import tabulate

table_data = []
for non_fine_tuned, fine_tuned in zip(base_results, fine_tuned_results):
    table_data.append({
        "Prompt": non_fine_tuned['prompt'],
        "Non-Finetuned Code": non_fine_tuned['code'],
        "Finetuned Code": fine_tuned['code']
    })
# Use tabulate for better formatting
df = pd.DataFrame(table_data)

# Print only the first 5 rows with tabulate
print(tabulate(df.head(), headers='keys', tablefmt='grid', numalign="left", stralign="left"))

### Salvataggio / caricamento dei risultati

**SAVE IN DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Save as JSON in Drive
with open("/content/drive/MyDrive/model_results.json", "w") as f:
    json.dump(results, f, indent=4)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAVE IN GIT**

In [ ]:
from google.colab import userdata
import os

# Install git (se non già installato)
!apt-get install git -qq

# Configura Git (modo corretto per Colab)
git_email = userdata.get('GitEmail')
git_username = userdata.get('GitUsername')
pat = userdata.get('PAT')

!git config --global user.email "{git_email}"
!git config --global user.name "{git_username}"

# Clona il repository (se non esiste già o forza il reclone)
if not os.path.exists('FineTuningAI'):
    !git clone https://{pat}@github.com/matteraggi/FineTuningAI.git
else:
    !rm -rf FineTuningAI
    !git clone https://{pat}@github.com/matteraggi/FineTuningAI.git

# Copia i risultati
!cp model_results.json FineTuningAI/results/

# Commit e push
%cd FineTuningAI
!git add results/model_results.json
!git commit -m "Save model results $(date +'%Y-%m-%d %H:%M:%S')"
!git push origin main
%cd ..

Cloning into 'FineTuningAI'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 241 (delta 9), reused 15 (delta 3), pack-reused 212 (from 1)
Receiving objects: 100% (241/241), 364.12 MiB | 33.41 MiB/s, done.
Resolving deltas: 100% (124/124), done.
Updating files: 100% (46/46), done.
/content/FineTuningAI
[main 424aab1] Save model results 2025-04-06 11:16:46
 1 file changed, 86 insertions(+), 86 deletions(-)
 rewrite results/model_results.json (98%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 4.15 KiB | 4.15 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/matteraggi/FineTuningAI.git
   57a9714..424aab1  main -> main
/content


**LOAD FROM DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import json

# Load JSON file from Drive
with open("/content/drive/MyDrive/model_results.json", "r") as f:
    results = json.load(f)

# Now results['base_results'] and results['fine_tuned_results'] hold the data
base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']


Mounted at /content/drive


**LOAD FROM GIT**

In [ ]:
import json
!apt-get install git
!git clone https://github.com/matteraggi/FineTuningAI.git

with open("FineTuningAI/results/model_results.json", "r") as f:
    results = json.load(f)

base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
fatal: destination path 'FineTuningAI' already exists and is not an empty directory.


### Confronto tra modelli ("*Complex checking*")

In [ ]:
!pip install pylint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.5/522.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.3/275.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 9.2 MB/s eta 0:00:00


**Confronto con SYSTEM_PROMPT + funzione**

In [ ]:
import ast
import radon.complexity
import subprocess
import json
import tempfile
import zss
import re
import os
import pandas as pd
from tabulate import tabulate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import torch
import random
import gc
from radon.complexity import cc_visit
import io
import sys
import difflib

# Configurazione
os.environ["HF_ALLOW_CODE_EVAL"] = "1"
base_model_name = "bigcode/starcoder2-3b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Caricamento del modello base
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
device_map = {"": device}
non_finetuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, device_map=device_map, quantization_config=bnb_config
)
non_finetuned_model.eval()

# Tokenizer
tokenizer_base = AutoTokenizer.from_pretrained(base_model_name)
tokenizer_base.pad_token = tokenizer_base.eos_token
tokenizer_base.padding_side = 'right'

SYSTEM_PROMPT = "Generate Python code for the following task:\n"

def estrai_docstring(code):
    match = re.search(r'("""|\'\'\')(.*?)(\1)', code, re.DOTALL)
    return match.group(2).strip() if match else ""

class CodeEvaluator:
    def __init__(self):
        self.tokenizer = None
        self.model = None

    def initialize_models(self):
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            base_model_name, device_map={"": device}, quantization_config=bnb_config
        ).eval()
    def run_pylint_analysis(self, code: str) -> dict:
      results = {
        "errors": 0,
        "warnings": 0,
        "conventions": 0,
        "refactors": 0,
        "messages": []
      }

      with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as temp_file:
        temp_file.write(code)
        temp_file_path = temp_file.name

      try:
        # Run pylint and capture the output
        completed = subprocess.run(
            ["pylint", "--disable=all", "--enable=E,W,C,R", "--score=n", temp_file_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        #print("--- STDOUT ---")
        #print(completed.stdout)
        #print("--- STDERR ---")
        #print(completed.stderr)
        # Parse stdout line by line
        for line in completed.stdout.splitlines():
          results["messages"].append(line)

          if re.search(r'\bE\d{4}\b', line):
            results["errors"] += 1
          elif re.search(r'\bW\d{4}\b', line):
            results["warnings"] += 1
          elif re.search(r'\bC\d{4}\b', line):
            results["conventions"] += 1
          elif re.search(r'\bR\d{4}\b', line):
            results["refactors"] += 1

      except Exception as e:
        results["messages"].append(f"Errore durante analisi pylint: {str(e)}")

      finally:
        os.remove(temp_file_path)

      return results


    def extract_pure_code(self, generated: str, prompt: str) -> str:
      """
      Rimuove il prompt ripetuto dal codice generato e isola solo la parte di codice Python.
      Supporta codice dentro
python ...
 oppure codice nudo.
      """
      # 1. Rimuove prompt (se presente all'inizio)
      generated = generated.strip()
      if generated.startswith(prompt.strip()):
        generated = generated[len(prompt):].strip()

      # 2. Estrae codice da blocchi markdown ```python
      match = re.search(r"```(?:python)?(.*?)```", generated, re.DOTALL)
      if match:
        return match.group(1).strip()

      # 3. Ritorna testo 'pulito'
      return generated.strip()

    def preprocess_code(self, code: str) -> str:
        if not code:
            return ""
        matches = re.findall(r'(def\s+\w+\(.*?\):.*?(?=\n\s*def\s|\Z))', code, re.DOTALL)
        code = matches[0] if matches else code
        code = re.sub(r'#.*', '', code)
        code = re.sub(r'\.\.\..*', '', code)
        code = re.sub(r'\s+\n', '\n', code)
        return code.strip()

    def is_valid_python(self, code: str) -> bool:
      try:
        ast.parse(code)
        return True
      except:
        return False
    def safe_parse(self, code):
      try:
        return ast.parse(code)
      except:
        return None
    def is_sum_function_correct(self, code: str) -> bool:
      try:
        tree = safe_parse(code)
        if not tree:
          return False
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef) and node.name == "sum":
                args = node.args.args
                if len(args) == 2:
                    for n in ast.walk(node):
                        if isinstance(n, ast.Return) and isinstance(n.value, ast.BinOp):
                            if isinstance(n.value.op, ast.Add):
                                return True
        return False
      except:
        return False
    def calculate_complexity(self, code: str) -> float:
      try:
        blocks = cc_visit(code)
        return sum(b.complexity for b in blocks) / len(blocks) if blocks else 0
      except:
        return 0


    def ast_to_node(self, node):
        if isinstance(node, ast.AST):
            children = [self.ast_to_node(child) for child in ast.iter_child_nodes(node)]
            return zss.Node(type(node).__name__, children)
        return zss.Node(str(node))

    def count_control_structures(self, code: str) -> int:
      try:
        tree = safe_parse(code)
        if not tree:
          return 0
        return sum(isinstance(n, (ast.If, ast.For, ast.While)) for n in ast.walk(tree))
      except:
        return 0

    def has_function_def(self, code: str) -> bool:
      try:
        tree = safe_parse(code)
        if not tree:
          return False
        return any(isinstance(n, ast.FunctionDef) for n in ast.walk(tree))
      except:
        return False
    def evaluate_code(self, code: str, canonical: str = None) -> dict:
      clean_code = code.strip()
      tree = self.safe_parse(clean_code)
      if tree:
        implementation_complete = any(isinstance(n, (ast.Return, ast.Yield, ast.Raise)) for n in ast.walk(tree))
      else:
        implementation_complete = False
      return {
        "syntactic_correct": self.is_valid_python(clean_code),
        "is_sum_function_correct": self.is_sum_function_correct(clean_code),
        "avg_complexity": self.calculate_complexity(clean_code),
        "code_length": len(clean_code),
        "has_function_def": self.has_function_def(clean_code),
        "has_return": "return" in clean_code.lower(),
        "implementation_complete": implementation_complete,
        "num_structures": self.count_control_structures(clean_code),
        **self.run_pylint_analysis(clean_code)
      }


    def generate_code(self, model, prompt, tokenizer):
        if not prompt.strip():
            return ""
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.8,
                top_p=0.95,
                repetition_penalty=1.1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)


    def run_evaluation(self):
        humaneval = load_dataset("openai/openai_humaneval", split="test")
        with open("fixed_sampled_humaneval.json", "r") as f:
            sampled_humaneval = json.load(f)

        self.initialize_models()
        results = {"base": [], "fine_tuned": []}

        for example in sampled_humaneval:
          prompt = example['prompt']
          canonical = example['canonical_solution']

          raw_base = self.generate_code(non_finetuned_model, SYSTEM_PROMPT + prompt, tokenizer_base)
          base_code = self.extract_pure_code(raw_base, prompt)
          base_metrics = self.evaluate_code(base_code, canonical)

          raw_finetuned = self.generate_code(finetuned_model, SYSTEM_PROMPT + prompt, tokenizer_finetuned)
          fine_tuned_code = self.extract_pure_code(raw_finetuned, prompt)
          fine_tuned_metrics = self.evaluate_code(fine_tuned_code, canonical)

          results["base"].append({
                "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                "code": base_code[:200] + "..." if len(base_code) > 200 else base_code,
                **base_metrics
            })

          results["fine_tuned"].append({
                "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                "code": fine_tuned_code[:200] + "..." if len(fine_tuned_code) > 200 else fine_tuned_code,
                **fine_tuned_metrics
          })
          gc.collect()

        self.analyze_results(results)

    def analyze_results(self, results):
        base_df = pd.DataFrame(results["base"])
        fine_tuned_df = pd.DataFrame(results["fine_tuned"])

        print("\n=== Metriche Aggregate ===")
        agg_metrics = [
            ["Sintatticamente Corretto", base_df["syntactic_correct"].mean(), fine_tuned_df["syntactic_correct"].mean()],
            ["Errori pylint", base_df["errors"].mean(), fine_tuned_df["errors"].mean()],
            ["Warning pylint", base_df["warnings"].mean(), fine_tuned_df["warnings"].mean()],
            ["Complessità Ciclomatica", base_df["avg_complexity"].mean(), fine_tuned_df["avg_complexity"].mean()],
            ["Implementazione Completa", base_df["implementation_complete"].mean(), fine_tuned_df["implementation_complete"].mean()]
        ]
        print(tabulate(agg_metrics, headers=["Metrica", "Base", "Fine-Tuned"], floatfmt=".2f", tablefmt="grid"))

        print("\n=== Esempi Dettagliati ===")
        valid_examples = 0
        for base, fine_tuned in zip(results["base"], results["fine_tuned"]):
          #if not base["syntactic_correct"] and not fine_tuned["syntactic_correct"]:
            #continue
          valid_examples += 1
          print(f"\nEsempio {valid_examples}:")
          print(f"Prompt: {base['prompt']}\n")

          print("🔹 Base Model:")
          print(f"Codice:\n{base['code']}")
          for k, v in base.items():
            if k not in ['prompt', 'code', 'errors', 'warnings', 'conventions', 'refactors']:
                print(f"- {k}: {v}")

          if "messages" in base:
            print("Messaggi Pylint:")
            for msg in base["messages"]:
              print(f"  • {msg}")

          print("\n🔸 Fine-Tuned Model:")
          print(f"Codice:\n{fine_tuned['code']}")
          for k, v in fine_tuned.items():
            if k not in ['prompt', 'code', 'errors', 'warnings', 'conventions', 'refactors']:
                print(f"- {k}: {v}")

          if "messages" in fine_tuned:
            print("Messaggi Pylint:")
            for msg in fine_tuned["messages"]:
              print(f"  • {msg}")

        if valid_examples == 0:
            print("Nessun esempio sintatticamente corretto trovato.")

if __name__ == "__main__":
    evaluator = CodeEvaluator()
    evaluator.run_evaluation()


=== Metriche Aggregate ===
+--------------------------+--------+--------------+
| Metrica                  |   Base |   Fine-Tuned |
+==========================+========+==============+
| Sintatticamente Corretto |   0.07 |         0.33 |
+--------------------------+--------+--------------+
| Errori pylint            |   0.93 |         1.03 |
+--------------------------+--------+--------------+
| Warning pylint           |   0.27 |         0.00 |
+--------------------------+--------+--------------+
| Complessità Ciclomatica  |   0.18 |         0.33 |
+--------------------------+--------+--------------+
| Implementazione Completa |   0.07 |         0.13 |
+--------------------------+--------+--------------+

=== Esempi Dettagliati ===

Esempio 1:
Prompt: 

def sort_even(l: list):
    """This function takes a list l and returns a list l' such that
    l'...

🔹 Base Model:
Codice:
Generate Python code for the following task:


def sort_even(l: list):
    """This function takes a list l a

**Confronto solo docstring**

In [ ]:
import ast
import radon.complexity
import subprocess
import json
import tempfile
import zss
import re
import os
import pandas as pd
from tabulate import tabulate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import torch
import random
import gc
from radon.complexity import cc_visit
import io
import sys
import difflib
# Configurazione
os.environ["HF_ALLOW_CODE_EVAL"] = "1"
base_model_name = "bigcode/starcoder2-3b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Caricamento del modello base
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
device_map = {"": device}
non_finetuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, device_map=device_map, quantization_config=bnb_config
)
non_finetuned_model.eval()

# Tokenizer
tokenizer_base = AutoTokenizer.from_pretrained(base_model_name)
tokenizer_base.pad_token = tokenizer_base.eos_token
tokenizer_base.padding_side = 'right'

SYSTEM_PROMPT = "Generate Python code for the following task:\n"



class CodeEvaluator:
    def __init__(self):
        self.tokenizer = None
        self.model = None
    def estrai_docstring(self, code):
      """Estrae il contenuto tra triple virgolette singole o doppie."""
      match = re.search(r'("""|\'\'\')(.*?)(\1)', code, re.DOTALL)
      return match.group(2).strip() if match else ""
    def initialize_models(self):
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            base_model_name, device_map={"": device}, quantization_config=bnb_config
        ).eval()
    def run_pylint_analysis(self, code: str) -> dict:
      results = {
        "errors": 0,
        "warnings": 0,
        "conventions": 0,
        "refactors": 0,
        "messages": []
      }

      with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as temp_file:
        temp_file.write(code)
        temp_file_path = temp_file.name

      try:
        # Run pylint and capture the output
        completed = subprocess.run(
            ["pylint", "--disable=all", "--enable=E,W,C,R", "--score=n", temp_file_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        #print("--- STDOUT ---")
        #print(completed.stdout)
        #print("--- STDERR ---")
        #print(completed.stderr)
        # Parse stdout line by line
        for line in completed.stdout.splitlines():
          results["messages"].append(line)

          if re.search(r'\bE\d{4}\b', line):
            results["errors"] += 1
          elif re.search(r'\bW\d{4}\b', line):
            results["warnings"] += 1
          elif re.search(r'\bC\d{4}\b', line):
            results["conventions"] += 1
          elif re.search(r'\bR\d{4}\b', line):
            results["refactors"] += 1

      except Exception as e:
        results["messages"].append(f"Errore durante analisi pylint: {str(e)}")

      finally:
        os.remove(temp_file_path)

      return results


    def extract_pure_code(self, generated: str, prompt: str) -> str:
      """
      Rimuove il prompt ripetuto dal codice generato e isola solo la parte di codice Python.
      Supporta codice dentro
python ...
 oppure codice nudo.
      """
      # 1. Rimuove prompt (se presente all'inizio)
      generated = generated.strip()
      if generated.startswith(prompt.strip()):
        generated = generated[len(prompt):].strip()

      # 2. Estrae codice da blocchi markdown ```python
      match = re.search(r"```(?:python)?(.*?)```", generated, re.DOTALL)
      if match:
        return match.group(1).strip()

      # 3. Ritorna testo 'pulito'
      return generated.strip()

    def preprocess_code(self, code: str) -> str:
        if not code:
            return ""
        matches = re.findall(r'(def\s+\w+\(.*?\):.*?(?=\n\s*def\s|\Z))', code, re.DOTALL)
        code = matches[0] if matches else code
        code = re.sub(r'#.*', '', code)
        code = re.sub(r'\.\.\..*', '', code)
        code = re.sub(r'\s+\n', '\n', code)
        return code.strip()

    def is_valid_python(self, code: str) -> bool:
      try:
        ast.parse(code)
        return True
      except:
        return False
    def safe_parse(self, code):
      try:
        return ast.parse(code)
      except:
        return None
    def is_sum_function_correct(self, code: str) -> bool:
      try:
        tree = safe_parse(code)
        if not tree:
          return False
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef) and node.name == "sum":
                args = node.args.args
                if len(args) == 2:
                    for n in ast.walk(node):
                        if isinstance(n, ast.Return) and isinstance(n.value, ast.BinOp):
                            if isinstance(n.value.op, ast.Add):
                                return True
        return False
      except:
        return False
    def calculate_complexity(self, code: str) -> float:
      try:
        blocks = cc_visit(code)
        return sum(b.complexity for b in blocks) / len(blocks) if blocks else 0
      except:
        return 0


    def ast_to_node(self, node):
        if isinstance(node, ast.AST):
            children = [self.ast_to_node(child) for child in ast.iter_child_nodes(node)]
            return zss.Node(type(node).__name__, children)
        return zss.Node(str(node))

    def count_control_structures(self, code: str) -> int:
      try:
        tree = safe_parse(code)
        if not tree:
          return 0
        return sum(isinstance(n, (ast.If, ast.For, ast.While)) for n in ast.walk(tree))
      except:
        return 0

    def has_function_def(self, code: str) -> bool:
      try:
        tree = safe_parse(code)
        if not tree:
          return False
        return any(isinstance(n, ast.FunctionDef) for n in ast.walk(tree))
      except:
        return False
    def evaluate_code(self, code: str, canonical: str = None) -> dict:
      clean_code = code.strip()
      tree = self.safe_parse(clean_code)
      if tree:
        implementation_complete = any(isinstance(n, (ast.Return, ast.Yield, ast.Raise)) for n in ast.walk(tree))
      else:
        implementation_complete = False
      return {
        "syntactic_correct": self.is_valid_python(clean_code),
        "is_sum_function_correct": self.is_sum_function_correct(clean_code),
        "avg_complexity": self.calculate_complexity(clean_code),
        "code_length": len(clean_code),
        "has_function_def": self.has_function_def(clean_code),
        "has_return": "return" in clean_code.lower(),
        "implementation_complete": implementation_complete,
        "num_structures": self.count_control_structures(clean_code),
        **self.run_pylint_analysis(clean_code)
      }


    def generate_code(self, model, prompt, tokenizer):
        if not prompt.strip():
            return ""
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.8,
                top_p=0.95,
                repetition_penalty=1.1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)


    def run_evaluation(self):
        humaneval = load_dataset("openai/openai_humaneval", split="test")
        with open("fixed_sampled_humaneval.json", "r") as f:
            sampled_humaneval = json.load(f)

        self.initialize_models()
        results = {"base": [], "fine_tuned": []}
        for example in sampled_humaneval:
          prompt = estrai_docstring(example["prompt"])
          canonical = example['canonical_solution']

          raw_base = self.generate_code(non_finetuned_model, SYSTEM_PROMPT + prompt, tokenizer_base)
          base_code = self.extract_pure_code(raw_base, prompt)
          base_metrics = self.evaluate_code(base_code, canonical)

          raw_finetuned = self.generate_code(finetuned_model, SYSTEM_PROMPT + prompt, tokenizer_finetuned)
          fine_tuned_code = self.extract_pure_code(raw_finetuned, prompt)
          fine_tuned_metrics = self.evaluate_code(fine_tuned_code, canonical)

          results["base"].append({
                "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                "code": base_code[:200] + "..." if len(base_code) > 200 else base_code,
                **base_metrics
            })

          results["fine_tuned"].append({
                "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                "code": fine_tuned_code[:200] + "..." if len(fine_tuned_code) > 200 else fine_tuned_code,
                **fine_tuned_metrics
          })
          gc.collect()

        self.analyze_results(results)

    def analyze_results(self, results):
        base_df = pd.DataFrame(results["base"])
        fine_tuned_df = pd.DataFrame(results["fine_tuned"])

        print("\n=== Metriche Aggregate ===")
        agg_metrics = [
            ["Sintatticamente Corretto", base_df["syntactic_correct"].mean(), fine_tuned_df["syntactic_correct"].mean()],
            ["Errori pylint", base_df["errors"].mean(), fine_tuned_df["errors"].mean()],
            ["Warning pylint", base_df["warnings"].mean(), fine_tuned_df["warnings"].mean()],
            ["Complessità Ciclomatica", base_df["avg_complexity"].mean(), fine_tuned_df["avg_complexity"].mean()],
            ["Implementazione Completa", base_df["implementation_complete"].mean(), fine_tuned_df["implementation_complete"].mean()]
        ]
        print(tabulate(agg_metrics, headers=["Metrica", "Base", "Fine-Tuned"], floatfmt=".2f", tablefmt="grid"))

        print("\n=== Esempi Dettagliati ===")
        valid_examples = 0
        for base, fine_tuned in zip(results["base"], results["fine_tuned"]):
          #if not base["syntactic_correct"] and not fine_tuned["syntactic_correct"]:
            #continue
          valid_examples += 1
          print(f"\nEsempio {valid_examples}:")
          print(f"Prompt: {base['prompt']}\n")

          print("🔹 Base Model:")
          print(f"Codice:\n{base['code']}")
          for k, v in base.items():
            if k not in ['prompt', 'code', 'errors', 'warnings', 'conventions', 'refactors']:
                print(f"- {k}: {v}")

          if "messages" in base:
            print("Messaggi Pylint:")
            for msg in base["messages"]:
              print(f"  • {msg}")

          print("\n🔸 Fine-Tuned Model:")
          print(f"Codice:\n{fine_tuned['code']}")
          for k, v in fine_tuned.items():
            if k not in ['prompt', 'code', 'errors', 'warnings', 'conventions', 'refactors']:
                print(f"- {k}: {v}")

          if "messages" in fine_tuned:
            print("Messaggi Pylint:")
            for msg in fine_tuned["messages"]:
              print(f"  • {msg}")

        if valid_examples == 0:
            print("Nessun esempio sintatticamente corretto trovato.")

if __name__ == "__main__":
    evaluator = CodeEvaluator()
    evaluator.run_evaluation()


=== Metriche Aggregate ===
+--------------------------+--------+--------------+
| Metrica                  |   Base |   Fine-Tuned |
+==========================+========+==============+
| Sintatticamente Corretto |   0.07 |         0.50 |
+--------------------------+--------+--------------+
| Errori pylint            |   0.93 |         0.70 |
+--------------------------+--------+--------------+
| Warning pylint           |   0.00 |         0.57 |
+--------------------------+--------+--------------+
| Complessità Ciclomatica  |   0.13 |         1.33 |
+--------------------------+--------+--------------+
| Implementazione Completa |   0.07 |         0.37 |
+--------------------------+--------+--------------+

=== Esempi Dettagliati ===

Esempio 1:
Prompt: This function takes a list l and returns a list l' such that
    l' is identical to l in the odd ind...

🔹 Base Model:
Codice:
Generate Python code for the following task:
This function takes a list l and returns a list l' such that
  